In [67]:
import torch
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import dotenv

import pandas as pd
import json
import pyarrow as pa
import pyarrow.dataset as ds
from datasets import Dataset


model_name = "serlikopar/Enlighten_Instruct"

In [68]:
dotenv.load_dotenv("hugging_face.env")

True

In [ ]:
secret_hf = os.getenv('HUGGINGFACE_TOKEN')
!huggingface-cli login --token $secret_hf

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [66]:
base_model = "mistralai/Mistral-7B-Instruct-v0.2"
new_model = "serlikopar/Enlighten_Instruct"
base_model_reload = AutoModelForCausalLM.from_pretrained(
        base_model,
        torch_dtype=torch.bfloat16,
        return_dict=True,
        low_cpu_mem_usage=True,
        device_map="auto",
        trust_remote_code=True,
)
model = PeftModel.from_pretrained(base_model_reload, new_model)
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

/opt/homebrew/Caskroom/mambaforge/base/envs/hertel_pc/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2.
401 Client Error. (Request ID: Root=1-664c7fa5-465e9f66371e321c05d68c65;7661e904-8ae5-470a-9c80-70dc782970fe)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.2 is restricted. You must be authenticated to access it.

In [21]:
def make_df_from_jsonl(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line_number, line in enumerate(file, start=1):
            try:
                json_data = json.loads(line)
                data.append(json_data)  # Append the entire JSON object
            except json.JSONDecodeError as e:
                print(f"Error parsing JSON on line {line_number}: {e}")
                continue

    print(f"Number of parsed objects: {len(data)}")  # Check how many items were added

    df = pd.DataFrame(data)

    if not df.empty:
        # If DataFrame is not empty, apply transformations
        df['messages'] = df['messages'].apply(
            lambda msgs: [{'role': msg['role'], 'content': msg['content'].replace('\\n', '\n')} for msg in msgs]
        )
    else:
        print("DataFrame is empty after creation.")

    return df

file_path = 'training_samples.jsonl'
df = make_df_from_jsonl(file_path)
print(df.head())


Number of parsed objects: 100
                                            messages
0  [{'role': 'system', 'content': 'Ich bin ein hi...
1  [{'role': 'system', 'content': 'Ich bin ein hi...
2  [{'role': 'system', 'content': 'Ich bin ein hi...
3  [{'role': 'system', 'content': 'Ich bin ein hi...
4  [{'role': 'system', 'content': 'Ich bin ein hi...


In [27]:
df

,messages
0,"[{'role': 'system', 'content': 'Ich bin ein hi..."
1,"[{'role': 'system', 'content': 'Ich bin ein hi..."
2,"[{'role': 'system', 'content': 'Ich bin ein hi..."
3,"[{'role': 'system', 'content': 'Ich bin ein hi..."
4,"[{'role': 'system', 'content': 'Ich bin ein hi..."
...,...
95,"[{'role': 'system', 'content': 'Ich bin ein hi..."
96,"[{'role': 'system', 'content': 'Ich bin ein hi..."
97,"[{'role': 'system', 'content': 'Ich bin ein hi..."
98,"[{'role': 'system', 'content': 'Ich bin ein hi..."


In [29]:
import pandas as pd
import json

def make_df_from_jsonl(file_path):
    questions = []
    answers = []

    with open(file_path, 'r', encoding='utf-8') as file:
        for line_number, line in enumerate(file, start=1):
            try:
                json_data = json.loads(line)
                # Extract messages
                messages = json_data['messages']
                # Initialize placeholders for user and assistant messages
                question = next((msg['content'] for msg in messages if msg['role'] == 'user'), None)
                answer = next((msg['content'] for msg in messages if msg['role'] == 'assistant'), None)
                
                # Only append if both question and answer are found
                if question and answer:
                    questions.append(question.replace('\\n', '\n'))
                    answers.append(answer.replace('\\n', '\n'))
                
            except json.JSONDecodeError as e:
                print(f"Error parsing JSON on line {line_number}: {e}")
                continue

    # Create DataFrame with specific columns
    df = pd.DataFrame({'Question': questions, 'Answer': answers})
    
    return df




In [45]:
file_path = 'training_samples.jsonl'
df = make_df_from_jsonl(file_path)

In [46]:
df.head()

,Question,Answer
0,Was besagt die Zustandsgleichung des idealen G...,Die Zustandsgleichung des idealen Gases lautet...
1,Was erklärt die kinetische Gastheorie über die...,"Die kinetische Gastheorie beschreibt, dass Gas..."
2,Was ist der Nullte Hauptsatz der Thermodynamik...,"Der Nullte Hauptsatz der Thermodynamik besagt,..."
3,Was ist der erste Hauptsatz der Thermodynamik?,Der erste Hauptsatz der Thermodynamik ist ein ...
4,Was ist der zweite Hauptsatz der Thermodynamik?,Der zweite Hauptsatz der Thermodynamik befasst...


In [47]:
df.columns

Index(['Question', 'Answer'], dtype='object')

In [58]:
df.to_csv("training_data_100_questions_with_formatting.csv", index=False)

In [49]:
df['text'] = '<s>[INST]' + df['Question'] +'[/INST]'+ df['Answer'] + '</s>'



In [ ]:
# remove columns
df=df.drop(['Question','Answer'],axis=1)

# convert to dataset object
dataset = ds.dataset(pa.Table.from_pandas(df).to_batches())
dataset = Dataset(pa.Table.from_pandas(df))

In [50]:
df

,Question,Answer,text
0,Was besagt die Zustandsgleichung des idealen G...,Die Zustandsgleichung des idealen Gases lautet...,<s>[INST]Was besagt die Zustandsgleichung des ...
1,Was erklärt die kinetische Gastheorie über die...,"Die kinetische Gastheorie beschreibt, dass Gas...",<s>[INST]Was erklärt die kinetische Gastheorie...
2,Was ist der Nullte Hauptsatz der Thermodynamik...,"Der Nullte Hauptsatz der Thermodynamik besagt,...",<s>[INST]Was ist der Nullte Hauptsatz der Ther...
3,Was ist der erste Hauptsatz der Thermodynamik?,Der erste Hauptsatz der Thermodynamik ist ein ...,<s>[INST]Was ist der erste Hauptsatz der Therm...
4,Was ist der zweite Hauptsatz der Thermodynamik?,Der zweite Hauptsatz der Thermodynamik befasst...,<s>[INST]Was ist der zweite Hauptsatz der Ther...
...,...,...,...
95,Was ist der Unterschied zwischen einem Katalys...,"Ein Katalysator ist eine Substanz, die die Ges...",<s>[INST]Was ist der Unterschied zwischen eine...
96,Was ist der Unterschied zwischen der Gibbs'sch...,Die Gibbs'sche freie Energie (G) wird bei kons...,<s>[INST]Was ist der Unterschied zwischen der ...
97,Was ist der Unterschied zwischen der inneren E...,Die innere Energie (U) ist die Gesamtenergie e...,<s>[INST]Was ist der Unterschied zwischen der ...
98,Was ist der Unterschied zwischen der kinetisch...,"Kinetische Energie ist die Energie, die ein Ob...",<s>[INST]Was ist der Unterschied zwischen der ...


In [52]:
df2 = df["text"]

In [55]:
df2 = pd.DataFrame(df2)

In [56]:
type(df2)

pandas.core.frame.DataFrame

In [57]:
df2

,text
0,<s>[INST]Was besagt die Zustandsgleichung des ...
1,<s>[INST]Was erklärt die kinetische Gastheorie...
2,<s>[INST]Was ist der Nullte Hauptsatz der Ther...
3,<s>[INST]Was ist der erste Hauptsatz der Therm...
4,<s>[INST]Was ist der zweite Hauptsatz der Ther...
...,...
95,<s>[INST]Was ist der Unterschied zwischen eine...
96,<s>[INST]Was ist der Unterschied zwischen der ...
97,<s>[INST]Was ist der Unterschied zwischen der ...
98,<s>[INST]Was ist der Unterschied zwischen der ...
